<a href="https://colab.research.google.com/github/soadD/netflix_data/blob/main/tabela_global_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Extração**

In [1]:
#Abertura das bibliotecas
import pandas as pd

In [2]:
# Importar a biblioteca para conectar o Google Drive ao ambiente Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Passo 4: Extração da informação via pandas
df = pd.read_csv('/content/drive/MyDrive/Soul Code/netflix/netflixtop10.csv')

## **Pré-Análise**

In [4]:
display(df)

,week,category,weekly_rank,show_title,season_title,weekly_hours_viewed,runtime,weekly_views,cumulative_weeks_in_top_10,is_staggered_launch,episode_launch_details
0,2024-05-26,Films (English),1,Atlas,NaN,56300000,2,28200000.0,1,False,NaN
1,2024-05-26,Films (English),2,Thelma the Unicorn,NaN,17400000,"1,6333",10700000.0,2,False,NaN
2,2024-05-26,Films (English),3,Ice Age: Dawn of the Dinosaurs,NaN,15100000,"1,5833",9500000.0,1,False,NaN
3,2024-05-26,Films (English),4,Mother of the Bride,NaN,14000000,"1,5",9300000.0,3,False,NaN
4,2024-05-26,Films (English),5,Disturbia,NaN,14100000,"1,75",8100000.0,2,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...
6075,2021-07-04,TV (Non-English),6,Elite,Elite: Season 1,10530000,NaN,NaN,1,False,NaN
6076,2021-07-04,TV (Non-English),7,Elite,Elite: Season 3,10200000,NaN,NaN,1,False,NaN
6077,2021-07-04,TV (Non-English),8,Elite,Elite: Season 2,10140000,NaN,NaN,1,False,NaN
6078,2021-07-04,TV (Non-English),9,Katla,Katla: Season 1,9190000,NaN,NaN,1,False,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6080 entries, 0 to 6079
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   week                        6080 non-null   object 
 1   category                    6080 non-null   object 
 2   weekly_rank                 6080 non-null   int64  
 3   show_title                  6080 non-null   object 
 4   season_title                2959 non-null   object 
 5   weekly_hours_viewed         6080 non-null   int64  
 6   runtime                     2000 non-null   object 
 7   weekly_views                2000 non-null   float64
 8   cumulative_weeks_in_top_10  6080 non-null   int64  
 9   is_staggered_launch         6080 non-null   bool   
 10  episode_launch_details      78 non-null     object 
dtypes: bool(1), float64(1), int64(3), object(6)
memory usage: 481.1+ KB


Colunas 'episode_launch_details','season_title','weekly_views' e 'runtime' não tem dados suficientes para fazer análises.

Coluna week é uma série temporal, porém está em object (string) ao invés de date time.


## **Transformação**

### **Exclusão de Colunas**

In [6]:
#Exclusão de colunas (números elevados de valores nulos)
lista = ['episode_launch_details','season_title','weekly_views','runtime','is_staggered_launch']

for i in lista:
  df.drop(i, axis=1, inplace=True)

### **Renomeação Colunas**

In [7]:
# Renomeando colunas para tradução
df.rename(columns={'week' : 'semana',
                   'category' : 'categoria',
                   'weekly_rank' : 'classificaçao_semanal',
                   'show_title' : 'titulo_do_programa/show',
                   'season_title' : 'titulo_da_temporada',
                   'weekly_hours_viewed' : 'horas_visualizadas',
                   'cumulative_weeks_in_top_10' : 'semanas_acumuladas_top10'}, inplace=True)

In [8]:
# Trocar valores especificos em colunas
df.loc[df.categoria == 'Films (English)', ['categoria']] = 'Filmes (Inglês)'
df.loc[df.categoria == 'Films (Non-English)', ['categoria']] = 'Filmes (Estrangeiros)'
df.loc[df.categoria == 'TV (English)', ['categoria']] = 'Séries de TV (Inglês)'
df.loc[df.categoria == 'TV (Non-English)', ['categoria']] = 'Séries de TV (Estrangeiros)'

### **Verificação de valores nulos**

In [9]:
# Verificando valores null
df.isnull().sum()

semana                      0
categoria                   0
classificaçao_semanal       0
titulo_do_programa/show     0
horas_visualizadas          0
semanas_acumuladas_top10    0
dtype: int64

In [10]:
# Verificando valores nan
df.isna().sum()

semana                      0
categoria                   0
classificaçao_semanal       0
titulo_do_programa/show     0
horas_visualizadas          0
semanas_acumuladas_top10    0
dtype: int64

### **Valores unicos e dados duplicados (REVISAR)**

Como em nossa tabela todos os valores de todas as colunas se repetem eventualmente, é necessário fazer uma verificação de valores duplicados de um mesmo título na mesma semana.

### **Conversão de String para Date Time**

In [11]:
df['semana'] = pd.to_datetime(df['semana'], infer_datetime_format=True)

<ipython-input-11-7c84dc2c8b97>:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['semana'] = pd.to_datetime(df['semana'], infer_datetime_format=True)


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6080 entries, 0 to 6079
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   semana                    6080 non-null   datetime64[ns]
 1   categoria                 6080 non-null   object        
 2   classificaçao_semanal     6080 non-null   int64         
 3   titulo_do_programa/show   6080 non-null   object        
 4   horas_visualizadas        6080 non-null   int64         
 5   semanas_acumuladas_top10  6080 non-null   int64         
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 285.1+ KB


In [13]:
df['mes'] = df['semana'].dt.month
df['ano'] = df['semana'].dt.year
df['dia'] = df['semana'].dt.day

In [14]:
# Trocar valores especificos em colunas
df.loc[df.mes == '1', ['mes']] = 'JAN'
df.loc[df.mes == '2', ['mes']] = 'FEV'
df.loc[df.mes == '3', ['mes']] = 'MAR'
df.loc[df.mes == '4', ['mes']] = 'ABR'
df.loc[df.mes == '5', ['mes']] = 'MAI'
df.loc[df.mes == '6', ['mes']] = 'JUN'
df.loc[df.mes == '7', ['mes']] = 'JUL'
df.loc[df.mes == '8', ['mes']] = 'AGO'
df.loc[df.mes == '9', ['mes']] = 'SET'
df.loc[df.mes == '10', ['mes']] = 'OUT'
df.loc[df.mes == '11', ['mes']] = 'NOV'
df.loc[df.mes == '12', ['mes']] = 'DEZ'

### **Traduzir valores das colunas ???????**

### **Verificação de Inconsistências**

In [15]:
#Verificando os tipos de objeto no dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6080 entries, 0 to 6079
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   semana                    6080 non-null   datetime64[ns]
 1   categoria                 6080 non-null   object        
 2   classificaçao_semanal     6080 non-null   int64         
 3   titulo_do_programa/show   6080 non-null   object        
 4   horas_visualizadas        6080 non-null   int64         
 5   semanas_acumuladas_top10  6080 non-null   int64         
 6   mes                       6080 non-null   object        
 7   ano                       6080 non-null   int32         
 8   dia                       6080 non-null   int32         
dtypes: datetime64[ns](1), int32(2), int64(3), object(3)
memory usage: 380.1+ KB


In [16]:
# Iterando sobre as colunas do DataFrame
for column in df.columns:

    # Verificando se a coluna contém valores inteiros
    if df[column].dtype == 'int':
        print(f"Valores inteiros únicos na coluna '{column}':")
        print(sorted(df[column].unique()))
        print("-" * 50)  # Linha de separação

    # Verificando se a coluna contém valores de ponto flutuante
    elif df[column].dtype == 'float':
        print(f"Valores decimais únicos na coluna '{column}':")
        print(sorted(df[column].unique()))
        print("-" * 50)  # Linha de separação

    # Verificando se a coluna contém valores de texto (string)
    elif df[column].dtype == 'object':
        print(f"Valores de texto únicos na coluna '{column}':")
        print(sorted(df[column].unique()))
        print("-" * 50)  # Linha de separação

Valores de texto únicos na coluna 'categoria':
['Filmes (Estrangeiros)', 'Filmes (Inglês)', 'Séries de TV (Estrangeiros)', 'Séries de TV (Inglês)']
--------------------------------------------------
Valores inteiros únicos na coluna 'classificaçao_semanal':
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
--------------------------------------------------
Valores de texto únicos na coluna 'titulo_do_programa/show':
["'83", '10 Days of a Bad Man', '10 Days of a Good Man', '1000 Miles from Christmas', '12 Strong', '13 Hours: The Secret Soldiers of Benghazi', '14 Peaks: Nothing Is Impossible', '1899', '1917', '2 Guns', '2 Hearts', '20th Century Girl', '21 Jump Street', '211', '24 Hours with Gaspar', '28 Days Haunted', '3 Body Problem', '365 Days', '365 Days: This Day', '4 Kings', '4 Kings 2', '40 Years Young', '42 Days of Darkness', '47 Meters Down: Uncaged', '47 Ronin', '6 Underground', '65', '6ixtynin9 The Series', '7 Prisoners', '7 Women and a Murder', '85 South: Ghetto Legends', 'A Bad Moms Christmas'

### **Verificação de Integridade**

In [17]:
#Instalação do pacote pandera
!pip install pandera

In [18]:
# importando o pacote pandera
import pandera as pa

In [19]:
#Alterando mes, ano e dia de Integer para String
df['mes'] = df['mes'].astype('str')
df['ano'] = df['ano'].astype('str')
df['dia'] = df['dia'].astype('str')

In [20]:
# Definição do esquema de validação
schema = pa.DataFrameSchema({'semana': pa.Column(pa.DateTime),
                             'categoria': pa.Column(pa.String),
                             'classificaçao_semanal': pa.Column(pa.Int),
                             'titulo_do_programa/show': pa.Column(pa.String),
                             'horas_visualizadas': pa.Column(pa.Int),
                             'semanas_acumuladas_top10': pa.Column(pa.Int),
                             'mes': pa.Column(pa.String),
                             'ano': pa.Column(pa.String),
                             'dia': pa.Column(pa.String)
                            })

In [21]:
df.dtypes

semana                      datetime64[ns]
categoria                           object
classificaçao_semanal                int64
titulo_do_programa/show             object
horas_visualizadas                   int64
semanas_acumuladas_top10             int64
mes                                 object
ano                                 object
dia                                 object
dtype: object

In [22]:
# Validar o DataFrame
schema.validate(df)

,semana,categoria,classificaçao_semanal,titulo_do_programa/show,horas_visualizadas,semanas_acumuladas_top10,mes,ano,dia
0,2024-05-26,Filmes (Inglês),1,Atlas,56300000,1,5,2024,26
1,2024-05-26,Filmes (Inglês),2,Thelma the Unicorn,17400000,2,5,2024,26
2,2024-05-26,Filmes (Inglês),3,Ice Age: Dawn of the Dinosaurs,15100000,1,5,2024,26
3,2024-05-26,Filmes (Inglês),4,Mother of the Bride,14000000,3,5,2024,26
4,2024-05-26,Filmes (Inglês),5,Disturbia,14100000,2,5,2024,26
...,...,...,...,...,...,...,...,...,...
6075,2021-07-04,Séries de TV (Estrangeiros),6,Elite,10530000,1,7,2021,4
6076,2021-07-04,Séries de TV (Estrangeiros),7,Elite,10200000,1,7,2021,4
6077,2021-07-04,Séries de TV (Estrangeiros),8,Elite,10140000,1,7,2021,4
6078,2021-07-04,Séries de TV (Estrangeiros),9,Katla,9190000,1,7,2021,4


## **Carregamento**

In [23]:
df.to_csv('/content/drive/MyDrive/Soul Code/netflix/netflixtop10_tratado.csv')